## Import Required Libraries

In [8]:
import os
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.optimizers import Adam
from joblib import dump
import numpy as np
import matplotlib.pyplot as plt
import pickle
from flask import Flask, request, jsonify
import requests

os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"


## Load the pre-trained model and add a classifier on top of the base model

In [9]:
model = Sequential([
    Conv2D(16, (3, 3), activation="relu", input_shape=(224, 224, 3)),
    MaxPooling2D(2, 2),
    Conv2D(32, (3, 3), activation="relu"),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation="relu"),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation="relu"),
    MaxPooling2D(2, 2),
    Conv2D(256, (3, 3), activation="relu"),
    MaxPooling2D(2, 2),
    Conv2D(512, (3, 3), activation="relu"),
    MaxPooling2D(2, 2)
    ])
model.add(Flatten())
model.add(Dense(512, activation="relu"))
model.add(Dense(256, activation="relu"))
model.add(Dense(128, activation="relu"))
model.add(Dense(100, activation="relu"))
model.add(Dense(80, activation="softmax"))



model.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 222, 222, 16)      448       
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 111, 111, 16)     0         
 g2D)                                                            
                                                                 
 conv2d_13 (Conv2D)          (None, 109, 109, 32)      4640      
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 54, 54, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_14 (Conv2D)          (None, 52, 52, 64)        18496     
                                                                 
 max_pooling2d_14 (MaxPoolin  (None, 26, 26, 64)      

## Define training parameters and callbacks

In [10]:
adam = Adam(learning_rate=0.001)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'], run_eagerly=True)
bs = 250
train_dir = './animals-detection-images-dataset/train'
test_dir = "./animals-detection-images-dataset/test"
train_datagen = ImageDataGenerator(rescale=1.0/255.)
test_datagen = ImageDataGenerator(rescale=1.0/255.)
train_generator = train_datagen.flow_from_directory(train_dir, batch_size=bs, class_mode='categorical', target_size=(224, 224), shuffle=True)
validation_generator = test_datagen.flow_from_directory(test_dir, batch_size=bs, class_mode='categorical', target_size=(224, 224), shuffle=True)




Found 22565 images belonging to 80 classes.
Found 6505 images belonging to 80 classes.


## Fit Model

In [11]:
history = model.fit(train_generator,
                              steps_per_epoch=train_generator.samples // bs,
                              epochs=15,
                              validation_data=validation_generator,
                              validation_steps=validation_generator.samples // bs)

Epoch 1/10
225/225 [==============================] - 1465s 7s/step - loss: 3.9759 - accuracy: 0.0791 - val_loss: 4.2817 - val_accuracy: 0.0260
Epoch 2/10
133/225 [================>.............] - ETA: 10:14 - loss: 3.8428 - accuracy: 0.0931

KeyboardInterrupt: 

## Evaluate model

In [ ]:
# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(validation_generator)

# Print the test loss and accuracy
print('Test loss:', test_loss)
print('Test accuracy:', test_acc)

train_loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(train_loss) + 1)

plt.plot(epochs, train_loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Plot the training and validation accuracy over epochs
train_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

plt.plot(epochs, train_acc, 'bo', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

## Save model

In [ ]:
model.save('./Animals_prediction_model.h5')
weights = model.get_weights()
dump(weights, './Animals_prediction_model.joblib')
with open('./Animals_prediction_model.pkl', 'wb') as f:
    pickle.dump(weights, f)

# Test prediction

In [ ]:
url = 'http://127.0.0.1:5050/predict'

predict_dir = './predict/1.jpg'
with open(predict_dir, 'rb') as img:
    response = requests.post(url, files={'file': img})
print(response.json())